In [1]:
import numpy as np
import pandas as pd
from sklearn import svm
import random
import copy
import matplotlib.pyplot as plt
import math
from sklearn.utils import shuffle
from tqdm import tqdm_notebook 

import cvxopt.solvers
import numpy.linalg as la
import logging

In [2]:
class Chromosome:
 
    def __init__(self,  vardim, bound):

        self.vardim = vardim
        self.bound = bound
        self.fitness = 0.
 
    def generate(self):

        len = self.vardim
        rnd1 = np.random.random(size=len)-0.5
        rnd2 = np.random.random(size=len)-0.5
        self.chrom = np.zeros(len)
        self.cig = np.zeros(len)
        for i in range(0, len):
            self.chrom[i] = self.bound[0, i] + \
                (self.bound[1, i] - self.bound[0, i]) * rnd1[i]
            self.cig[i] = self.bound[0, i] + \
                (self.bound[1, i] - self.bound[0, i]) * rnd2[i]
        #print("chorm: {}, ".format(self.chrom)+"cigmal: {}".format(self.cig))
    def calculateFitness(self,dataset):
        
        #print(self.chrom)
        self.fitness = SVMResult(
            self.vardim, self.chrom, self.bound, dataset)
        
    def print_(self):
        
        print(self.chrom)

In [3]:
class GA:
 
    def __init__(self, sizepop, vardim, bound, MAXGEN, params, k):
        
        self.remain = sizepop
        self.realsize = self.remain
        self.sizepop = self.remain*k
        self.MAXGEN = MAXGEN
        self.vardim = vardim
        self.bound = bound
        self.k = k
        self.population = []
        self.fitness = np.zeros((self.sizepop, 1))
        self.trace = np.zeros((self.MAXGEN, 2))
        self.params = params
        self.dataset = self.creat_dataset(pd.read_csv("diabetes.csv"))
        self.lr = 1
        
    def creat_dataset(self, df):
        
        df = shuffle(df)
        size = len(df)
        df['split'] = 0
        df.iloc[0:math.ceil(0.3*size),-1] = 'train'
        #df.iloc[math.ceil(0.7*size):math.ceil(0.85*size), -1] = 'test'
        df.iloc[math.ceil(0.7*size):size, -1] = 'val' #调高验证集比例

        return df
 
    def initialize(self):
        
        for i in range(0, self.remain):
            ind = Chromosome(self.vardim, self.bound)
            ind.generate()
            self.population = np.append(self.population, ind)
        self.population = np.array(self.population) 
        
    def evaluate(self):
        
        fitness = []
        for i in tqdm_notebook(range(0, self.realsize)):
            self.population[i].calculateFitness(self.dataset)
            #print("###/n")
            fitness.append(self.population[i].fitness)
        self.fitness = np.array(fitness)
        best_idx = np.argmax(self.fitness)
        self.best_score = self.fitness[best_idx]
        self.best = self.population[best_idx]
        
    def show(self):
        
        for i in range(0, self.realsize):
            #print('chorme {}: '.format(i))
            self.population[i].print_()
            print(self.population[i].fitness)
 
    def solve(self):
        
        self.t = 0
        self.initialize()
        self.evaluate()
        #best = np.max(self.fitness)
        #bestIndex = np.argmax(self.fitness)
        #self.best = copy.deepcopy(self.population[bestIndex])
        #self.avefitness = np.mean(self.fitness)
        #self.trace[self.t, 0] = self.best.fitness
        #self.trace[self.t, 1] = self.avefitness
        #self.trace[self.t, 0] = (1 - self.best.fitness) / self.best.fitness
        #self.trace[self.t, 1] = (1 - self.avefitness) / self.avefitness
        print("本代最好的染色体: {}".format(self.best.chrom))
        print("本代最高分: {}".format(self.best_score))
        #print("第%d代: 最高分: %f; 平均分%f" % (
            #self.t, self.trace[self.t, 0], self.trace[self.t, 1]))
        while (self.t < self.MAXGEN - 1):
            self.t += 1
            self.crossover()
            self.mutation()
            self.evaluate()
            self.selection()
            
            #print("best? {}".format(self.population[-1].chrom))
            #self.show()
            best = np.max(self.fitness)
            bestIndex = np.argmax(self.fitness)
            #print("本代最好的染色体: {}".format(self.population[bestIndex].chrom))
            #print("本代最高分: {}".format(self.population[bestIndex].fitness))
            #if best > self.best.fitness:#改动
            #self.best = copy.deepcopy(self.population[bestIndex])
            #self.avefitness = np.mean(self.fitness)
            #self.trace[self.t, 0] = self.best.fitness
            #self.trace[self.t, 1] = self.avefitness
            #self.trace[self.t, 0] = (1 - self.best.fitness) / self.best.fitness
            #self.trace[self.t, 1] = (1 - self.avefitness) / self.avefitness
            
            #print("第%d代: 最高分: %f; 平均分%f" % (
                #self.t, self.trace[self.t, 0], self.trace[self.t, 1]))
            print("本代最好的染色体: {}".format(self.population[bestIndex].chrom))
            print("本代最高分: {}".format(self.population[bestIndex].fitness))
 
        print("Optimal function value is: %f; " %
              self.trace[self.t, 0])
        print("Optimal solution is:")
        print(self.best.chrom)
        self.printResult()
 
    def selection(self):
        
        #硬选择
        #print("max: {}".format(np.argmax(self.fitness)))
        #print(self.fitness[-1])
        sort_idx = np.argsort(self.fitness, axis=0).reshape(1,-1)[0]  #????
        top_idx = sort_idx[len(self.fitness)-self.remain-1:-1]
        #print(sort_idx[0:self.remain])
        new_fitness = copy.deepcopy(self.fitness[top_idx])
        self.fitness = new_fitness
        #print(len(self.fitness))
        new = copy.deepcopy(self.population[top_idx]) #深复制
        self.population  = new
        self.realsize = len(self.population)
        #print(self.realsize)
    
    """
     def selection(self):
     
        #软选择
        #print("max: {}".format(np.argmax(self.fitness)))
        #print(self.fitness)
        sort_idx = np.argsort(self.fitness, axis=0).reshape(1,-1)[0]
        #print(sort_idx[0:self.remain])
        new = copy.deepcopy(self.population[sort_idx[0:self.remain]])
        self.population  = new
        self.realsize = len(self.population)
        #print(self.realsize)
    """
    
    def crossover(self):
        
        newpop = []
       # print(self.sizepop-self.remain)
        #print(len(self.population)) 
        #print(self.sizepop-self.remain-10)
        for i in range(self.sizepop-self.remain-10):
            idx1 = random.randint(0, self.remain - 1)
            idx2 = random.randint(0, self.remain - 1)
            while idx2 == idx1:
                idx2 = random.randint(0, self.remain - 1)
            new = copy.deepcopy(self.population[idx1])
            new.chrom += self.population[idx2].chrom
            new.chrom /= 2
            new.cig += self.population[idx2].cig
            new.cig /= 2
            self.population = np.append(self.population, new)
        #print(len(self.population))
        for i in range(9):
            new = Chromosome(self.vardim, self.bound)
            new.generate()
            self.population = np.append(self.population, new)
        self.realsize = len(self.population)
        #print(self.realsize)
        
    def mutation(self):
        
        newpop = []
        self.lr *= 0.9
        for i in range(0, self.sizepop-1):
            p = random.random()-0.5
            newpop.append(copy.deepcopy(self.population[i]))
            rand = random.random()-0.5
            for j in range(0, self.vardim):
                #s = 0
                #while newpop[i].chrom[j]<=self.bound[0][j] or newpop[i].chrom[j]>=self.bound[1][j]: #限制bound
                    
                #s+=1
                randn = random.random()-0.5
                flag = 1
                while newpop[i].cig[j]==0 or flag==1:
                    flag = 0
                    newpop[i].cig[j] = newpop[i].cig[j]*np.exp(rand+randn)

                newpop[i].chrom[j] = newpop[i].cig[j]+newpop[i].cig[j]*randn*self.lr #增加学习率
        newpop.append(self.best)    #保留最佳    
                #print(s)
        self.population = np.array(newpop)
        self.realsize = len(self.population)
        #print(self.realsize)
 
    def printResult(self):
        
        x = np.arange(0, self.MAXGEN)
        y1 = self.trace[:, 0]
        y2 = self.trace[:, 1]
        plt.plot(x, y1, 'r', label='optimal value')
        plt.plot(x, y2, 'g', label='average value')
        plt.xlabel("Iteration")
        plt.ylabel("function value")
        plt.title("Genetic algorithm for function optimization")
        plt.legend()
        plt.show()


In [76]:
MIN_SUPPORT_VECTOR_MULTIPLIER = 1e-5

class SVM(object):
    def __init__(self, kernel, c):
        self._kernel = kernel
        self._c = c

    def train(self, X, y):
        
        lagrange_multipliers = self._compute_multipliers(X, y)
        return self._construct_predictor(X, y, lagrange_multipliers)

    def _gram_matrix(self, X):
        n_samples, n_features = X.shape
        K = np.zeros((n_samples, n_samples))
        
        for i, x_i in enumerate(X):
            for j, x_j in enumerate(X):
                K[i, j] = self._kernel(x_i, x_j)
        return K

    def _construct_predictor(self, X, y, lagrange_multipliers):
        support_vector_indices = \
            lagrange_multipliers > MIN_SUPPORT_VECTOR_MULTIPLIER

        support_multipliers = lagrange_multipliers[support_vector_indices]
        support_vectors = X[support_vector_indices]
        support_vector_labels = y[support_vector_indices]
        
        bias = np.mean(
            [y_k - SVMPredictor(
                kernel=self._kernel,
                bias=0.0,
                weights=support_multipliers,
                support_vectors=support_vectors,
                support_vector_labels=support_vector_labels).predict(x_k)
             for (y_k, x_k) in zip(support_vector_labels, support_vectors)])

        return SVMPredictor(
            kernel=self._kernel,
            bias=bias,
            weights=support_multipliers,
            support_vectors=support_vectors,
            support_vector_labels=support_vector_labels)

    def _compute_multipliers(self, X, y):
        n_samples, n_features = X.shape

        K = self._gram_matrix(X)
        
        P = cvxopt.matrix(np.outer(y, y) * K)
        q = cvxopt.matrix(-1 * np.ones(n_samples))
        
        G_std = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
        h_std = cvxopt.matrix(np.zeros(n_samples))

        G_slack = cvxopt.matrix(np.diag(np.ones(n_samples)))
        h_slack = cvxopt.matrix(np.ones(n_samples) * self._c)

        G = cvxopt.matrix(np.vstack((G_std, G_slack)))
        h = cvxopt.matrix(np.vstack((h_std, h_slack)))
        A = cvxopt.matrix(np.double(y), (1, n_samples))
        b = cvxopt.matrix(0.0)
        
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)

        return np.ravel(solution['x'])


class SVMPredictor(object):
    def __init__(self,
                 kernel,
                 bias,
                 weights,
                 support_vectors,
                 support_vector_labels):
        self._kernel = kernel
        self._bias = bias
        self._weights = weights         #shape = (sv_num, )
        self._support_vectors = support_vectors       #shape = (sv_num, feature, )
        self._support_vector_labels = support_vector_labels    #shape = (sv_num, )
        self._s = np.array([alpha*y for alpha,y in zip(self._weights,self._support_vector_labels)]) #shape = (sv_num, )

    def predict(self, x):
        
        result = self._bias
        for z_i, x_i, y_i in zip(self._weights,
                                 self._support_vectors,
                                 self._support_vector_labels):
            result += z_i * y_i * self._kernel(x_i, x)
        return np.sign(result).item()
    
    def predict_vec(self, X):
        """
        X.shape = (batch, feature)
        """
        
        k = np.array([[self._kernel(x_i, X_j) for x_i in self._support_vectors] for X_j in X])  #shape = (batch, sv_num, )
        print(self._s.reshape(-1,1).shape)
        score = np.dot(k, self._s.reshape(-1,1))  #shape = (batch, )
        print(score.shape)
        return np.sign(score.reshape(-1,1))
            
        

In [77]:
class Kernel(object):
    @staticmethod
    def linear():
        return lambda x, y: np.inner(x, y)

    @staticmethod
    def gaussian(sigma):
        return lambda x, y: \
            np.exp(-np.sqrt(la.norm(x-y) ** 2 / (2 * sigma ** 2)))

    @staticmethod
    def _polykernel(dimension, offset):
        return lambda x, y: (offset + np.inner(x, y)) ** dimension

    @classmethod
    def inhomogenous_polynomial(cls, dimension):
        return cls._polykernel(dimension=dimension, offset=1.0)

    @classmethod
    def homogenous_polynomial(cls, dimension):
        return cls._polykernel(dimension=dimension, offset=0.0)

    @staticmethod
    def hyperbolic_tangent(kappa, c):
        return lambda x, y: np.tanh(kappa * np.dot(x, y) + c)

    @staticmethod
    def radial_basis(gamma=10):
        return lambda x, y: np.exp(-gamma*la.norm(np.subtract(x, y)))

In [80]:
def score(y_bar, val_y):
    
    Error = 0
    for i in range(len(y_bar)):
        miss=abs(y_bar[i-1]-val_y[i-1])
        Error += miss
        
    return Error / len(y_bar)
 
def SVMResult(vardim, x, bound, dataset):
    
    X = dataset.loc[dataset['split'] == 'train'].iloc[:,0:-2].values
    y = dataset.loc[dataset['split'] == 'train'].iloc[:,-2].values
    val_X = dataset.loc[dataset['split'] == 'val'].iloc[:,0:-2].values
    val_y = dataset.loc[dataset['split'] == 'val'].iloc[:,-2].values
    c = x[0]
    g = x[1]
    #f = x[2]#四参数
    rbf_kernel = Kernel.radial_basis(gamma=abs(g))
    svm = SVM(rbf_kernel, abs(c))
    predictor = svm.train(X, y)
    y_bar = predictor.predict_vec(val_X)
    #predictval=clf.predict(val_X)
    #print(predictval)
    #return msefunc(predictval,val_y) 
    return score(y_bar, val_y)


In [ ]:
bound = np.array([[0,0],[20, 1]])
ga = GA(50, 2, bound, 50, [0.5, 0.1, 0.5], 7)#调高种群大小， 调低生存比 //又调回来了
ga.solve()

W:\conda\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


     pcost       dcost       gap    pres   dres
 0: -7.7614e+02 -9.7009e+03  1e+04  1e-01  2e-15
 1: -8.6678e+02 -1.5304e+03  7e+02  7e-03  2e-15
 2: -1.4011e+03 -1.4508e+03  5e+01  3e-04  2e-15
 3: -1.4062e+03 -1.4067e+03  5e-01  3e-06  1e-15
 4: -1.4063e+03 -1.4063e+03  5e-03  3e-08  1e-15
 5: -1.4063e+03 -1.4063e+03  5e-05  3e-10  1e-15
Optimal solution found.
(146, 1)
(230, 1)
     pcost       dcost       gap    pres   dres
 0: -6.0413e+02 -6.0762e+03  8e+03  2e-01  2e-15
 1: -6.8304e+02 -1.1448e+03  5e+02  8e-03  1e-15
 2: -1.0586e+03 -1.0862e+03  3e+01  2e-04  7e-16
 3: -1.0622e+03 -1.0625e+03  3e-01  2e-06  1e-15
 4: -1.0623e+03 -1.0623e+03  3e-03  2e-08  1e-15
 5: -1.0623e+03 -1.0623e+03  3e-05  2e-10  9e-16
Optimal solution found.
(146, 1)
(230, 1)
     pcost       dcost       gap    pres   dres
 0: -2.0633e+02 -7.9876e+02  1e+03  8e-01  7e-16
 1: -2.0993e+02 -3.5407e+02  1e+02  8e-03  3e-16
 2: -2.6394e+02 -2.6800e+02  4e+00  2e-04  4e-16
 3: -2.6663e+02 -2.6667e+02  4e-02  2

 3: -6.2681e+02 -6.2690e+02  9e-02  6e-07  4e-16
 4: -6.2683e+02 -6.2683e+02  9e-04  6e-09  6e-16
 5: -6.2683e+02 -6.2683e+02  9e-06  6e-11  7e-16
Optimal solution found.
(146, 1)
(230, 1)
     pcost       dcost       gap    pres   dres
 0: -7.0581e+02 -8.1208e+03  1e+04  2e-01  2e-15
 1: -7.9221e+02 -1.3721e+03  6e+02  8e-03  1e-15
 2: -1.2610e+03 -1.3011e+03  4e+01  3e-04  1e-15
 3: -1.2656e+03 -1.2660e+03  4e-01  3e-06  1e-15
 4: -1.2656e+03 -1.2656e+03  4e-03  3e-08  2e-15
 5: -1.2656e+03 -1.2656e+03  4e-05  3e-10  1e-15
Optimal solution found.
(146, 1)
(230, 1)
     pcost       dcost       gap    pres   dres
 0: -1.6570e+02 -5.0374e+02  1e+03  1e+00  6e-16
 1: -1.5124e+02 -2.6681e+02  1e+02  1e-02  4e-16
 2: -1.8241e+02 -1.8689e+02  5e+00  4e-04  3e-16
 3: -1.8537e+02 -1.8541e+02  5e-02  4e-06  3e-16
 4: -1.8540e+02 -1.8540e+02  5e-04  4e-08  2e-16
 5: -1.8540e+02 -1.8540e+02  5e-06  4e-10  2e-16
Optimal solution found.
(146, 1)
(230, 1)
     pcost       dcost       gap    pres   

(146, 1)
(230, 1)
     pcost       dcost       gap    pres   dres
 0: -5.0058e+02 -4.2851e+03  6e+03  2e-01  2e-15
 1: -5.6948e+02 -9.1489e+02  4e+02  8e-03  8e-16
 2: -8.5236e+02 -8.6940e+02  2e+01  2e-04  7e-16
 3: -8.5513e+02 -8.5530e+02  2e-01  2e-06  1e-15
 4: -8.5516e+02 -8.5516e+02  2e-03  2e-08  9e-16
 5: -8.5516e+02 -8.5516e+02  2e-05  2e-10  9e-16
Optimal solution found.
(146, 1)
(230, 1)
     pcost       dcost       gap    pres   dres
 0: -7.4760e+02 -9.0432e+03  1e+04  2e-01  2e-15
 1: -8.3659e+02 -1.4660e+03  7e+02  7e-03  8e-16
 2: -1.3442e+03 -1.3900e+03  5e+01  3e-04  7e-16
 3: -1.3491e+03 -1.3496e+03  5e-01  3e-06  2e-15
 4: -1.3492e+03 -1.3492e+03  5e-03  3e-08  9e-16
 5: -1.3492e+03 -1.3492e+03  5e-05  3e-10  1e-15
Optimal solution found.
(146, 1)
(230, 1)
     pcost       dcost       gap    pres   dres
 0: -1.1749e+02 -2.3986e+02  1e+03  3e+00  4e-16
 1: -7.3220e+01 -1.6761e+02  1e+02  3e-02  6e-16
 2: -8.5698e+01 -9.1084e+01  6e+00  2e-03  3e-16
 3: -8.8939e+01 -8.

     pcost       dcost       gap    pres   dres
 0: -1.7288e+02 -5.5260e+02  1e+03  1e+00  4e-16
 1: -1.6188e+02 -2.8299e+02  1e+02  1e-02  5e-16
 2: -1.9670e+02 -2.0124e+02  5e+00  4e-04  2e-16
 3: -1.9973e+02 -1.9978e+02  5e-02  4e-06  4e-16
 4: -1.9976e+02 -1.9976e+02  5e-04  4e-08  4e-16
 5: -1.9976e+02 -1.9976e+02  5e-06  4e-10  3e-16
Optimal solution found.
(146, 1)
(230, 1)
     pcost       dcost       gap    pres   dres
 0: -9.3651e+02 -1.3810e+04  2e+04  1e-01  3e-15
 1: -1.0348e+03 -1.8949e+03  9e+02  6e-03  1e-15
 2: -1.7203e+03 -1.7947e+03  8e+01  3e-04  2e-15
 3: -1.7270e+03 -1.7277e+03  8e-01  3e-06  1e-15
 4: -1.7270e+03 -1.7270e+03  8e-03  3e-08  3e-15
 5: -1.7270e+03 -1.7270e+03  8e-05  3e-10  1e-15
Optimal solution found.
(146, 1)
(230, 1)
     pcost       dcost       gap    pres   dres
 0: -4.0550e+02 -2.8991e+03  4e+03  3e-01  1e-15
 1: -4.6188e+02 -7.0470e+02  3e+02  5e-03  1e-15
 2: -6.6298e+02 -6.7251e+02  1e+01  8e-05  8e-16
 3: -6.6497e+02 -6.6507e+02  1e-01  8

...

 1: -3.9603e+02 -6.0885e+02  2e+02  4e-03  8e-16
 2: -5.5251e+02 -5.6109e+02  9e+00  9e-05  8e-16
 3: -5.5405e+02 -5.5413e+02  9e-02  9e-07  8e-16
 4: -5.5406e+02 -5.5406e+02  9e-04  9e-09  4e-16
 5: -5.5406e+02 -5.5406e+02  9e-06  9e-11  1e-15
Optimal solution found.
(146, 1)
(230, 1)
     pcost       dcost       gap    pres   dres
 0: -3.0688e+02 -1.7233e+03  3e+03  4e-01  1e-15
 1: -3.4281e+02 -5.3844e+02  2e+02  4e-03  5e-16
 2: -4.6653e+02 -4.7333e+02  7e+00  1e-04  7e-16
 3: -4.6775e+02 -4.6782e+02  7e-02  1e-06  5e-16
 4: -4.6776e+02 -4.6776e+02  7e-04  1e-08  8e-16
 5: -4.6776e+02 -4.6776e+02  7e-06  1e-10  6e-16
Optimal solution found.
(146, 1)
(230, 1)
     pcost       dcost       gap    pres   dres
 0: -3.7316e+02 -2.4842e+03  4e+03  3e-01  1e-15
 1: -4.2388e+02 -6.4516e+02  2e+02  3e-03  1e-15
 2: -5.9858e+02 -6.0738e+02  9e+00  8e-05  7e-16
 3: -6.0031e+02 -6.0039e+02  9e-02  8e-07  8e-16
 4: -6.0032e+02 -6.0032e+02  9e-04  8e-09  6e-16
 5: -6.0032e+02 -6.0032e+02  9e-06  

 3: -3.7701e+03 -3.7728e+03  3e+00  1e-06  4e-15
 4: -3.7703e+03 -3.7703e+03  3e-02  1e-08  4e-15
 5: -3.7703e+03 -3.7703e+03  3e-04  1e-10  4e-15
Optimal solution found.
(146, 1)
(230, 1)
     pcost       dcost       gap    pres   dres
 0: -3.4755e+02 -2.1759e+03  3e+03  4e-01  1e-15
 1: -3.9302e+02 -6.0491e+02  2e+02  4e-03  6e-16
 2: -5.4757e+02 -5.5610e+02  9e+00  1e-04  5e-16
 3: -5.4909e+02 -5.4918e+02  9e-02  1e-06  7e-16
 4: -5.4911e+02 -5.4911e+02  9e-04  1e-08  6e-16
 5: -5.4911e+02 -5.4911e+02  9e-06  1e-10  4e-16
Optimal solution found.
(146, 1)
(230, 1)
     pcost       dcost       gap    pres   dres
 0: -1.7937e+02 -5.9796e+02  1e+03  1e+00  5e-16
 1: -1.7139e+02 -2.9731e+02  1e+02  1e-02  3e-16
 2: -2.0966e+02 -2.1421e+02  5e+00  3e-04  3e-16
 3: -2.1271e+02 -2.1275e+02  5e-02  3e-06  3e-16
 4: -2.1274e+02 -2.1274e+02  5e-04  3e-08  3e-16
 5: -2.1274e+02 -2.1274e+02  5e-06  3e-10  5e-16
Optimal solution found.
(146, 1)
(230, 1)
     pcost       dcost       gap    pres   

 5: -1.7540e+01 -1.7540e+01  3e-06  9e-09  2e-16
Optimal solution found.
(146, 1)
(230, 1)
     pcost       dcost       gap    pres   dres
 0: -2.3815e+02 -1.0613e+03  2e+03  6e-01  9e-16
 1: -2.5363e+02 -4.1638e+02  2e+02  6e-03  5e-16
 2: -3.2905e+02 -3.3150e+02  3e+00  6e-05  4e-16
 3: -3.3028e+02 -3.3031e+02  3e-02  6e-07  4e-16
 4: -3.3029e+02 -3.3029e+02  3e-04  6e-09  4e-16
Optimal solution found.
(146, 1)
(230, 1)
     pcost       dcost       gap    pres   dres
 0: -8.2964e+02 -1.0993e+04  1e+04  1e-01  2e-15
 1: -9.2311e+02 -1.6514e+03  8e+02  6e-03  1e-15
 2: -1.5076e+03 -1.5652e+03  6e+01  3e-04  1e-15
 3: -1.5132e+03 -1.5138e+03  6e-01  3e-06  2e-15
 4: -1.5133e+03 -1.5133e+03  6e-03  3e-08  1e-15
 5: -1.5133e+03 -1.5133e+03  6e-05  3e-10  1e-15
Optimal solution found.
(146, 1)
(230, 1)
     pcost       dcost       gap    pres   dres
 0: -1.4891e+02 -3.9507e+02  9e+02  1e+00  5e-16
 1: -1.2589e+02 -2.2746e+02  1e+02  1e-02  3e-16
 2: -1.4914e+02 -1.5330e+02  4e+00  5e-04  6

 3: -1.4718e+03 -1.4723e+03  6e-01  3e-06  2e-15
 4: -1.4718e+03 -1.4718e+03  6e-03  3e-08  1e-15
 5: -1.4718e+03 -1.4718e+03  6e-05  3e-10  1e-15
Optimal solution found.
(146, 1)
(230, 1)
     pcost       dcost       gap    pres   dres
 0: -2.6449e+02 -1.2998e+03  2e+03  5e-01  5e-16
 1: -2.8856e+02 -4.6491e+02  2e+02  5e-03  5e-16
 2: -3.8205e+02 -3.8580e+02  4e+00  8e-05  5e-16
 3: -3.8298e+02 -3.8301e+02  4e-02  8e-07  5e-16
 4: -3.8298e+02 -3.8299e+02  4e-04  8e-09  6e-16
Optimal solution found.
(146, 1)
(230, 1)
     pcost       dcost       gap    pres   dres
 0: -1.3205e+03 -2.6515e+04  3e+04  8e-02  6e-15
 1: -1.4305e+03 -2.7916e+03  1e+03  4e-03  3e-15
 2: -2.4847e+03 -2.6268e+03  1e+02  2e-04  3e-15
 3: -2.4949e+03 -2.4964e+03  1e+00  2e-06  2e-15
 4: -2.4950e+03 -2.4951e+03  1e-02  2e-08  3e-15
 5: -2.4950e+03 -2.4950e+03  1e-04  2e-10  2e-15
Optimal solution found.
(146, 1)
(230, 1)
     pcost       dcost       gap    pres   dres
 0: -6.2541e+02 -6.4806e+03  8e+03  2e-01  2

(146, 1)
(230, 1)
     pcost       dcost       gap    pres   dres
 0: -8.3446e+02 -1.1114e+04  1e+04  1e-01  3e-15
 1: -9.2817e+02 -1.6623e+03  8e+02  6e-03  1e-15
 2: -1.5172e+03 -1.5755e+03  6e+01  3e-04  2e-15
 3: -1.5229e+03 -1.5234e+03  6e-01  3e-06  2e-15
 4: -1.5229e+03 -1.5229e+03  6e-03  3e-08  1e-15
 5: -1.5229e+03 -1.5229e+03  6e-05  3e-10  2e-15
Optimal solution found.
(146, 1)
(230, 1)
     pcost       dcost       gap    pres   dres
 0: -2.4010e+02 -1.0783e+03  2e+03  6e-01  7e-16
 1: -2.5625e+02 -4.2006e+02  2e+02  6e-03  5e-16
 2: -3.3299e+02 -3.3549e+02  3e+00  6e-05  5e-16
 3: -3.3418e+02 -3.3421e+02  3e-02  6e-07  5e-16
 4: -3.3419e+02 -3.3419e+02  3e-04  6e-09  5e-16
Optimal solution found.
(146, 1)
(230, 1)
     pcost       dcost       gap    pres   dres
 0: -1.1234e+02 -2.1370e+02  1e+03  3e+00  3e-16
 1: -6.4664e+01 -1.5485e+02  1e+02  3e-02  4e-16
 2: -7.5460e+01 -8.0823e+01  6e+00  2e-03  4e-16
 3: -7.8639e+01 -7.8694e+01  6e-02  2e-05  2e-16
 4: -7.8670e+01 -7.

In [ ]:
print(ga.trace[t, 0])

In [ ]:
def creat_dataset1(df):
        df = shuffle(df)
        size = len(df)
        df['split'] = 0
        df.iloc[0:math.ceil(0.3*size),-1] = 'train'
        df.iloc[math.ceil(0.7*size):, -1] = 'val'
        #df.iloc[math.ceil(0.85*size):size, -1] = 'val'

        return df
dataset = creat_dataset1(pd.read_csv("diabetes.csv"))

In [ ]:
bound = np.tile([[0], [1]], 2) 
bound

In [115]:
def SVMResult1(dataset):
    X = dataset.loc[dataset['split'] == 'train'].iloc[:,0:-2].values
    y = dataset.loc[dataset['split'] == 'train'].iloc[:,-2].values
    val_X = dataset.loc[dataset['split'] == 'val'].iloc[:,0:-2].values
    val_y = dataset.loc[dataset['split'] == 'val'].iloc[:,-2].values
    
    clf = svm.SVC(C=1.21619432, gamma=0.00701896, kernel='rbf')
    clf.fit(X, y)
    #predictval=clf.predict(val_X)
    #print(predictval)
    #return msefunc(predictval,val_y) 
    return clf.score(val_X, val_y)
print(SVMResult1(dataset))

0.6565217391304348


In [104]:
a = [1,2,3,45,5,6]

In [ ]:
a[[2,3,4,5]]

In [2]:
import cvxopt.solvers